<a href="https://colab.research.google.com/github/BebishaC/Empathetic-AI-Chatbot-with-Real-Time-Emotion-Detection/blob/main/Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =====================================
# 1️⃣ Import Libraries
# =====================================
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    pipeline
)
import json
import random
import re
from datetime import datetime
from pathlib import Path

# =====================================
# 2️⃣ Load Emotion Detection Model
# =====================================
print("📦 Loading emotion detection model...")
emotion_pipeline = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    device=0 if torch.cuda.is_available() else -1
)

def detect_emotion(text):
    """Detect emotion from text with confidence score"""
    try:
        if not text or len(text.strip()) == 0:
            return "neutral", 0.0
        result = emotion_pipeline(text[:512])[0]
        return result['label'].lower(), result['score']
    except Exception as e:
        return "neutral", 0.0

# =====================================
# 3️⃣ Load Conversational Model
# =====================================
print("📦 Loading conversational model...")

model_name = "facebook/blenderbot-400M-distill"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

print("✅ Model loaded successfully!\n")

# =====================================
# 4️⃣ Emotion-Aware Response Components
# =====================================

EMOTION_STARTERS = {
    "joy": [
        "That's wonderful news!",
        "I'm so happy for you!",
        "Congratulations!",
        "That's fantastic!",
        "How exciting!",
        "That's amazing!",
        "What great news!"
    ],
    "sadness": [
        "I'm sorry to hear that.",
        "That sounds really difficult.",
        "I can hear the pain in your words.",
        "That must be so hard.",
        "I'm here for you.",
        "That sounds tough to deal with."
    ],
    "anger": [
        "I can hear your frustration.",
        "That sounds really upsetting.",
        "I understand why you're angry.",
        "That must be frustrating.",
        "Your anger is valid.",
        "That's a difficult situation."
    ],
    "fear": [
        "That sounds scary.",
        "I hear your concerns.",
        "That's a lot to worry about.",
        "Your fear is understandable.",
        "That must feel overwhelming.",
        "I understand your anxiety."
    ],
    "surprise": [
        "Wow, that's unexpected!",
        "What a surprise!",
        "That must have caught you off guard!",
        "That's quite a development!",
        "I can imagine that was surprising!"
    ],
    "love": [
        "That's beautiful!",
        "How wonderful!",
        "That's so heartwarming!",
        "What a lovely feeling!",
        "That sounds special!"
    ],
    "neutral": [
        "I hear you.",
        "I understand.",
        "Thanks for sharing that.",
        "I'm listening.",
        "Tell me more."
    ]
}

EMOTION_QUESTIONS = {
    "joy": [
        "Tell me more about it!",
        "What made this moment so special?",
        "How are you celebrating?",
        "When did you find out?",
        "How do you feel about it?",
        "What happens next?"
    ],
    "sadness": [
        "Would you like to talk about it?",
        "What's been the hardest part?",
        "How long have you been feeling this way?",
        "Is there anything that might help?",
        "What would make things better?"
    ],
    "anger": [
        "What happened?",
        "Do you want to talk about it?",
        "What upset you the most?",
        "How can I help?",
        "What would make this better?"
    ],
    "fear": [
        "What are you most worried about?",
        "What would help you feel safer?",
        "Have you dealt with this before?",
        "What's your biggest concern?",
        "How can I support you?"
    ],
    "surprise": [
        "How are you feeling about it?",
        "What happened?",
        "Was it a good surprise?",
        "What was your first reaction?",
        "How unexpected was this?"
    ],
    "love": [
        "Tell me more about this!",
        "What makes this so special?",
        "How long have you felt this way?",
        "What do you love most about it?"
    ],
    "neutral": [
        "What's on your mind?",
        "How are you doing?",
        "What else would you like to share?",
        "How do you feel about that?"
    ]
}

# =====================================
# 5️⃣ Response Validation
# =====================================

def is_valid_response(response, user_input):
    """Check if model response is appropriate"""

    if not response or len(response.strip()) < 5:
        return False

    response_lower = response.lower()

    # Bad patterns indicating confused responses
    bad_patterns = [
        "i'll try to keep that in mind",
        "i'll keep that in mind",
        "i will try to",
        "that's a good way to look at it",
        "that is a great way to look at it",
        "i appreciate your",
        "thank you for",
        "thanks for",
        "that's good advice",
        "i agree with you",
        "you're right about that"
    ]

    for pattern in bad_patterns:
        if pattern in response_lower:
            return False

    return True

# =====================================
# 6️⃣ Hybrid Response Generator
# =====================================

def generate_model_continuation(user_input, emotion_starter, conversation_history):
    """Try to generate a continuation after emotion acknowledgment"""

    try:
        # Build minimal context
        context = ""
        if conversation_history and len(conversation_history) > 2:
            recent_user = [m for m in conversation_history[-4:] if m['sender'] == 'user']
            if recent_user:
                context = f"Context: {recent_user[-1]['message']}. "

        # Simple prompt focused on natural continuation
        prompt = f"{context}Person: {user_input}\nYou: {emotion_starter}"

        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            max_length=256,
            truncation=True
        ).to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=80,
                min_length=10,
                num_beams=3,
                temperature=0.7,
                do_sample=True,
                repetition_penalty=1.5,
                no_repeat_ngram_size=2
            )

        continuation = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

        # Remove the prompt part if it's repeated
        if continuation.startswith(emotion_starter):
            continuation = continuation[len(emotion_starter):].strip()

        # Validate
        if is_valid_response(continuation, user_input):
            return continuation

        return None

    except Exception as e:
        return None

def generate_emotion_aware_response(user_input, emotion, confidence, conversation_history):
    """Generate response with strong emotion acknowledgment"""

    # Always start with appropriate emotion acknowledgment
    starter = random.choice(EMOTION_STARTERS.get(emotion, EMOTION_STARTERS["neutral"]))

    # For high confidence emotions, try to get model continuation
    continuation = None
    if confidence > 0.7 and random.random() > 0.3:
        continuation = generate_model_continuation(user_input, starter, conversation_history)

    # Build final response
    if continuation and len(continuation.split()) > 3:
        # Use model continuation
        response = f"{starter} {continuation}"
    else:
        # Use question-based response
        question = random.choice(EMOTION_QUESTIONS.get(emotion, EMOTION_QUESTIONS["neutral"]))

        # Sometimes add context awareness
        if len(conversation_history) > 2 and random.random() > 0.5:
            response = f"{starter} {question}"
        else:
            response = f"{starter} {question}"

    # Clean up
    response = response.strip()

    # Ensure proper ending
    if response and response[-1] not in '.!?':
        if '?' not in response:
            response += '.'

    return response

# =====================================
# 7️⃣ Special Case Handlers
# =====================================

def handle_greeting(user_input):
    """Handle greetings specially"""
    greetings = ["hi", "hii", "hello", "hey", "hiya", "greetings"]

    if user_input.lower().strip() in greetings:
        responses = [
            "Hello! How are you doing today?",
            "Hi there! What's on your mind?",
            "Hey! How can I support you today?",
            "Hello! I'm here to listen. How are you feeling?",
            "Hi! What would you like to talk about?"
        ]
        return random.choice(responses)

    return None

def handle_short_input(user_input, conversation_history):
    """Handle very short inputs contextually"""

    if len(user_input.split()) <= 2 and conversation_history:
        # Get last bot message to understand context
        last_bot = None
        for msg in reversed(conversation_history):
            if msg['sender'] == 'bot':
                last_bot = msg['message']
                break

        # If bot asked a question, acknowledge the answer
        if last_bot and '?' in last_bot:
            acknowledgments = [
                "I see.",
                "Got it.",
                "Understood.",
                "Thank you for sharing that.",
                "I appreciate you telling me that."
            ]
            return random.choice(acknowledgments)

    return None

# =====================================
# 8️⃣ Main Response Generator
# =====================================

def generate_response(user_input, conversation_history):
    """Main response generation pipeline"""

    # Handle greetings
    greeting_response = handle_greeting(user_input)
    if greeting_response:
        return "neutral", 0.9, greeting_response

    # Detect emotion
    emotion, confidence = detect_emotion(user_input)

    # Handle very short responses with context
    if len(user_input.split()) <= 3 and conversation_history:
        short_response = handle_short_input(user_input, conversation_history)
        if short_response:
            # Still generate follow-up
            question = random.choice(EMOTION_QUESTIONS.get(emotion, EMOTION_QUESTIONS["neutral"]))
            response = f"{short_response} {question}"
            return emotion, confidence, response

    # Generate emotion-aware response
    response = generate_emotion_aware_response(user_input, emotion, confidence, conversation_history)

    return emotion, confidence, response

# =====================================
# 9️⃣ Conversation Manager
# =====================================

class ConversationManager:
    """Manages conversation history and saving"""

    def __init__(self, save_dir="conversations"):
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(exist_ok=True)
        self.conversation_history = []
        self.start_time = datetime.now()
        self.emotions_detected = {}
        self.emotion_confidence = []

    def add_message(self, sender, message, emotion=None, confidence=None):
        """Add message to history"""
        self.conversation_history.append({
            "timestamp": datetime.now().isoformat(),
            "sender": sender,
            "message": message,
            "emotion": emotion,
            "confidence": confidence
        })

        if emotion:
            self.emotions_detected[emotion] = self.emotions_detected.get(emotion, 0) + 1

        if confidence is not None:
            self.emotion_confidence.append(confidence)

    def save_conversation(self):
        """Save conversation to JSON file"""
        if not self.conversation_history:
            print("⚠️ No conversation to save.")
            return None

        filename = self.start_time.strftime("%Y%m%d_%H%M%S") + "_conversation.json"
        filepath = self.save_dir / filename

        avg_confidence = sum(self.emotion_confidence) / len(self.emotion_confidence) if self.emotion_confidence else 0

        conversation_data = {
            "metadata": {
                "start_time": self.start_time.isoformat(),
                "end_time": datetime.now().isoformat(),
                "total_messages": len(self.conversation_history),
                "user_messages": len([m for m in self.conversation_history if m["sender"] == "user"]),
                "bot_messages": len([m for m in self.conversation_history if m["sender"] == "bot"]),
                "emotions_detected": self.emotions_detected,
                "avg_emotion_confidence": round(avg_confidence, 3)
            },
            "conversation": self.conversation_history
        }

        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(conversation_data, f, indent=2, ensure_ascii=False)

        print(f"\n✅ Conversation saved to: {filepath}")
        return filepath

    def print_statistics(self):
        """Print conversation statistics"""
        if not self.conversation_history:
            print("⚠️ No conversation data.")
            return

        avg_confidence = sum(self.emotion_confidence) / len(self.emotion_confidence) if self.emotion_confidence else 0

        print("\n" + "="*70)
        print("📊 CONVERSATION STATISTICS")
        print("="*70)
        print(f"Total Messages: {len(self.conversation_history)}")
        print(f"User Messages: {len([m for m in self.conversation_history if m['sender'] == 'user'])}")
        print(f"Bot Messages: {len([m for m in self.conversation_history if m['sender'] == 'bot'])}")
        print(f"Duration: {(datetime.now() - self.start_time).total_seconds():.1f} seconds")
        print(f"Avg Emotion Confidence: {avg_confidence:.1%}")

        if self.emotions_detected:
            print(f"\n📈 Emotions Detected:")
            for emotion, count in sorted(self.emotions_detected.items(), key=lambda x: x[1], reverse=True):
                print(f"  • {emotion.capitalize()}: {count} times")
        print("="*70 + "\n")

# =====================================
# 🔟 Chat Interface
# =====================================

def chat():
    """Multi-turn conversation interface"""
    print("\n" + "="*70)
    print("💬 EMPATHETIC AI CHATBOT v5.0")
    print("="*70)
    print("🤖 Emotion-First Response System")
    print("💝 BlenderBot + Curated Empathetic Templates")
    print("🎯 Context-Aware & Emotionally Intelligent")
    print("="*70)
    print("\nCommands:")
    print("  • 'quit' or 'exit' - Save and exit")
    print("  • 'clear' - Reset conversation")
    print("  • 'history' - Show conversation")
    print("  • 'save' - Save conversation")
    print("  • 'stats' - Show statistics")
    print("\n" + "="*70 + "\n")

    manager = ConversationManager()

    while True:
        try:
            user_input = input("👤 You: ").strip()

            if not user_input:
                continue

            if user_input.lower() in ["quit", "exit"]:
                manager.print_statistics()
                manager.save_conversation()
                print("👋 Thank you for sharing with me. Take care!\n")
                break

            if user_input.lower() == "clear":
                manager = ConversationManager()
                print("🔄 Conversation cleared. Let's start fresh!\n")
                continue

            if user_input.lower() == "save":
                manager.save_conversation()
                continue

            if user_input.lower() == "stats":
                manager.print_statistics()
                continue

            if user_input.lower() == "history":
                print("\n" + "="*70)
                print("📜 CONVERSATION HISTORY")
                print("="*70)
                for msg in manager.conversation_history:
                    sender = "👤 You" if msg["sender"] == "user" else "🤖 Bot"
                    emotion_info = ""
                    if msg.get("emotion"):
                        conf = f"{msg['confidence']:.0%}" if msg.get('confidence') else ""
                        emotion_info = f" [{msg['emotion']} {conf}]" if conf else f" [{msg['emotion']}]"
                    print(f"{sender}{emotion_info}: {msg['message']}")
                print("="*70 + "\n")
                continue

            # Add user message
            manager.add_message("user", user_input)

            # Generate response
            emotion, confidence, response = generate_response(
                user_input,
                manager.conversation_history
            )

            # Add bot response
            manager.add_message("bot", response, emotion, confidence)

            # Display response
            confidence_bar = "🔴" if confidence < 0.6 else "🟡" if confidence < 0.8 else "🟢"
            print(f"🤖 Bot [{emotion} {confidence_bar} {confidence:.0%}]: {response}\n")

        except KeyboardInterrupt:
            print("\n\n" + "="*70)
            manager.print_statistics()
            manager.save_conversation()
            print("👋 Take care!\n")
            break
        except Exception as e:
            print(f"⚠️ Error: {e}\n")
            continue

# =====================================
# Main
# =====================================

if __name__ == "__main__":
    chat()

In [ ]:
# =====================================
# 1️⃣ Import Libraries
# =====================================
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    pipeline
)
import json
import random
import re
from datetime import datetime
from pathlib import Path

# =====================================
# 2️⃣ Load Emotion Detection Model
# =====================================
print("📦 Loading emotion detection model...")
emotion_pipeline = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    device=0 if torch.cuda.is_available() else -1
)

def detect_emotion(text):
    """Detect emotion from text with confidence score"""
    try:
        if not text or len(text.strip()) == 0:
            return "neutral", 0.0
        result = emotion_pipeline(text[:512])[0]
        return result['label'].lower(), result['score']
    except Exception as e:
        return "neutral", 0.0

# =====================================
# 3️⃣ Load Conversational Model
# =====================================
print("📦 Loading conversational model...")

model_name = "facebook/blenderbot-400M-distill"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

print("✅ Model loaded successfully!\n")

# =====================================
# 4️⃣ Emotion-Aware Response Components
# =====================================

EMOTION_STARTERS = {
    "joy": [
        "That's wonderful news!",
        "I'm so happy for you!",
        "Congratulations!",
        "That's fantastic!",
        "How exciting!",
        "That's amazing!",
        "What great news!"
    ],
    "sadness": [
        "I'm sorry to hear that.",
        "That sounds really difficult.",
        "I can hear the pain in your words.",
        "That must be so hard.",
        "I'm here for you.",
        "That sounds tough to deal with."
    ],
    "anger": [
        "I can hear your frustration.",
        "That sounds really upsetting.",
        "I understand why you're angry.",
        "That must be frustrating.",
        "Your anger is valid.",
        "That's a difficult situation."
    ],
    "fear": [
        "That sounds scary.",
        "I hear your concerns.",
        "That's a lot to worry about.",
        "Your fear is understandable.",
        "That must feel overwhelming.",
        "I understand your anxiety."
    ],
    "surprise": [
        "Wow, that's unexpected!",
        "What a surprise!",
        "That must have caught you off guard!",
        "That's quite a development!",
        "I can imagine that was surprising!"
    ],
    "love": [
        "That's beautiful!",
        "How wonderful!",
        "That's so heartwarming!",
        "What a lovely feeling!",
        "That sounds special!"
    ],
    "neutral": [
        "I hear you.",
        "I understand.",
        "Thanks for sharing that.",
        "I'm listening.",
        "Tell me more."
    ]
}

EMOTION_QUESTIONS = {
    "joy": [
        "Tell me more about it!",
        "What made this moment so special?",
        "How are you celebrating?",
        "When did you find out?",
        "How do you feel about it?",
        "What happens next?"
    ],
    "sadness": [
        "Would you like to talk about it?",
        "What's been the hardest part?",
        "How long have you been feeling this way?",
        "Is there anything that might help?",
        "What would make things better?"
    ],
    "anger": [
        "What happened?",
        "Do you want to talk about it?",
        "What upset you the most?",
        "How can I help?",
        "What would make this better?"
    ],
    "fear": [
        "What are you most worried about?",
        "What would help you feel safer?",
        "Have you dealt with this before?",
        "What's your biggest concern?",
        "How can I support you?"
    ],
    "surprise": [
        "How are you feeling about it?",
        "What happened?",
        "Was it a good surprise?",
        "What was your first reaction?",
        "How unexpected was this?"
    ],
    "love": [
        "Tell me more about this!",
        "What makes this so special?",
        "How long have you felt this way?",
        "What do you love most about it?"
    ],
    "neutral": [
        "What's on your mind?",
        "How are you doing?",
        "What else would you like to share?",
        "How do you feel about that?"
    ]
}

# =====================================
# 5️⃣ Response Validation
# =====================================

def is_valid_response(response, user_input):
    """Check if model response is appropriate"""

    if not response or len(response.strip()) < 5:
        return False

    response_lower = response.lower()

    # Bad patterns indicating confused responses
    bad_patterns = [
        "i'll try to keep that in mind",
        "i'll keep that in mind",
        "i will try to",
        "that's a good way to look at it",
        "that is a great way to look at it",
        "i appreciate your",
        "thank you for",
        "thanks for",
        "that's good advice",
        "i agree with you",
        "you're right about that"
    ]

    for pattern in bad_patterns:
        if pattern in response_lower:
            return False

    return True

# =====================================
# 6️⃣ Hybrid Response Generator
# =====================================

def generate_model_continuation(user_input, emotion_starter, conversation_history):
    """Try to generate a continuation after emotion acknowledgment"""

    try:
        # Build minimal context
        context = ""
        if conversation_history and len(conversation_history) > 2:
            recent_user = [m for m in conversation_history[-4:] if m['sender'] == 'user']
            if recent_user:
                context = f"Context: {recent_user[-1]['message']}. "

        # Simple prompt focused on natural continuation
        prompt = f"{context}Person: {user_input}\nYou: {emotion_starter}"

        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            max_length=256,
            truncation=True
        ).to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=80,
                min_length=10,
                num_beams=3,
                temperature=0.7,
                do_sample=True,
                repetition_penalty=1.5,
                no_repeat_ngram_size=2
            )

        continuation = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

        # Remove the prompt part if it's repeated
        if continuation.startswith(emotion_starter):
            continuation = continuation[len(emotion_starter):].strip()

        # Validate
        if is_valid_response(continuation, user_input):
            return continuation

        return None

    except Exception as e:
        return None

def generate_emotion_aware_response(user_input, emotion, confidence, conversation_history):
    """Generate response with strong emotion acknowledgment"""

    # Always start with appropriate emotion acknowledgment
    starter = random.choice(EMOTION_STARTERS.get(emotion, EMOTION_STARTERS["neutral"]))

    # For high confidence emotions, try to get model continuation
    continuation = None
    if confidence > 0.7 and random.random() > 0.3:
        continuation = generate_model_continuation(user_input, starter, conversation_history)

    # Build final response
    if continuation and len(continuation.split()) > 3:
        # Use model continuation
        response = f"{starter} {continuation}"
    else:
        # Use question-based response
        question = random.choice(EMOTION_QUESTIONS.get(emotion, EMOTION_QUESTIONS["neutral"]))

        # Sometimes add context awareness
        if len(conversation_history) > 2 and random.random() > 0.5:
            response = f"{starter} {question}"
        else:
            response = f"{starter} {question}"

    # Clean up
    response = response.strip()

    # Ensure proper ending
    if response and response[-1] not in '.!?':
        if '?' not in response:
            response += '.'

    return response

# =====================================
# 7️⃣ Special Case Handlers
# =====================================

def handle_greeting(user_input):
    """Handle greetings specially"""
    greetings = ["hi", "hii", "hello", "hey", "hiya", "greetings"]

    if user_input.lower().strip() in greetings:
        responses = [
            "Hello! How are you doing today?",
            "Hi there! What's on your mind?",
            "Hey! How can I support you today?",
            "Hello! I'm here to listen. How are you feeling?",
            "Hi! What would you like to talk about?"
        ]
        return random.choice(responses)

    return None

def handle_short_input(user_input, conversation_history):
    """Handle very short inputs contextually"""

    if len(user_input.split()) <= 2 and conversation_history:
        # Get last bot message to understand context
        last_bot = None
        for msg in reversed(conversation_history):
            if msg['sender'] == 'bot':
                last_bot = msg['message']
                break

        # If bot asked a question, acknowledge the answer
        if last_bot and '?' in last_bot:
            acknowledgments = [
                "I see.",
                "Got it.",
                "Understood.",
                "Thank you for sharing that.",
                "I appreciate you telling me that."
            ]
            return random.choice(acknowledgments)

    return None

# =====================================
# 8️⃣ Main Response Generator
# =====================================

def generate_response(user_input, conversation_history):
    """Main response generation pipeline"""

    # Handle greetings
    greeting_response = handle_greeting(user_input)
    if greeting_response:
        return "neutral", 0.9, greeting_response

    # Detect emotion
    emotion, confidence = detect_emotion(user_input)

    # Handle very short responses with context
    if len(user_input.split()) <= 3 and conversation_history:
        short_response = handle_short_input(user_input, conversation_history)
        if short_response:
            # Still generate follow-up
            question = random.choice(EMOTION_QUESTIONS.get(emotion, EMOTION_QUESTIONS["neutral"]))
            response = f"{short_response} {question}"
            return emotion, confidence, response

    # Generate emotion-aware response
    response = generate_emotion_aware_response(user_input, emotion, confidence, conversation_history)

    return emotion, confidence, response

# =====================================
# 9️⃣ Conversation Manager
# =====================================

class ConversationManager:
    """Manages conversation history and saving"""

    def __init__(self, save_dir="conversations"):
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(exist_ok=True)
        self.conversation_history = []
        self.start_time = datetime.now()
        self.emotions_detected = {}
        self.emotion_confidence = []

    def add_message(self, sender, message, emotion=None, confidence=None):
        """Add message to history"""
        self.conversation_history.append({
            "timestamp": datetime.now().isoformat(),
            "sender": sender,
            "message": message,
            "emotion": emotion,
            "confidence": confidence
        })

        if emotion:
            self.emotions_detected[emotion] = self.emotions_detected.get(emotion, 0) + 1

        if confidence is not None:
            self.emotion_confidence.append(confidence)

    def save_conversation(self):
        """Save conversation to JSON file"""
        if not self.conversation_history:
            print("⚠️ No conversation to save.")
            return None

        filename = self.start_time.strftime("%Y%m%d_%H%M%S") + "_conversation.json"
        filepath = self.save_dir / filename

        avg_confidence = sum(self.emotion_confidence) / len(self.emotion_confidence) if self.emotion_confidence else 0

        conversation_data = {
            "metadata": {
                "start_time": self.start_time.isoformat(),
                "end_time": datetime.now().isoformat(),
                "total_messages": len(self.conversation_history),
                "user_messages": len([m for m in self.conversation_history if m["sender"] == "user"]),
                "bot_messages": len([m for m in self.conversation_history if m["sender"] == "bot"]),
                "emotions_detected": self.emotions_detected,
                "avg_emotion_confidence": round(avg_confidence, 3)
            },
            "conversation": self.conversation_history
        }

        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(conversation_data, f, indent=2, ensure_ascii=False)

        print(f"\n✅ Conversation saved to: {filepath}")
        return filepath

    def print_statistics(self):
        """Print conversation statistics"""
        if not self.conversation_history:
            print("⚠️ No conversation data.")
            return

        avg_confidence = sum(self.emotion_confidence) / len(self.emotion_confidence) if self.emotion_confidence else 0

        print("\n" + "="*70)
        print("📊 CONVERSATION STATISTICS")
        print("="*70)
        print(f"Total Messages: {len(self.conversation_history)}")
        print(f"User Messages: {len([m for m in self.conversation_history if m['sender'] == 'user'])}")
        print(f"Bot Messages: {len([m for m in self.conversation_history if m['sender'] == 'bot'])}")
        print(f"Duration: {(datetime.now() - self.start_time).total_seconds():.1f} seconds")
        print(f"Avg Emotion Confidence: {avg_confidence:.1%}")

        if self.emotions_detected:
            print(f"\n📈 Emotions Detected:")
            for emotion, count in sorted(self.emotions_detected.items(), key=lambda x: x[1], reverse=True):
                print(f"  • {emotion.capitalize()}: {count} times")
        print("="*70 + "\n")

# =====================================
# 🔟 Chat Interface
# =====================================

def chat():
    """Multi-turn conversation interface"""
    print("\n" + "="*70)
    print("💬 EMPATHETIC AI CHATBOT v5.0")
    print("="*70)
    print("🤖 Emotion-First Response System")
    print("💝 BlenderBot + Curated Empathetic Templates")
    print("🎯 Context-Aware & Emotionally Intelligent")
    print("="*70)
    print("\nCommands:")
    print("  • 'quit' or 'exit' - Save and exit")
    print("  • 'clear' - Reset conversation")
    print("  • 'history' - Show conversation")
    print("  • 'save' - Save conversation")
    print("  • 'stats' - Show statistics")
    print("\n" + "="*70 + "\n")

    manager = ConversationManager()

    while True:
        try:
            user_input = input("👤 You: ").strip()

            if not user_input:
                continue

            if user_input.lower() in ["quit", "exit"]:
                manager.print_statistics()
                manager.save_conversation()
                print("👋 Thank you for sharing with me. Take care!\n")
                break

            if user_input.lower() == "clear":
                manager = ConversationManager()
                print("🔄 Conversation cleared. Let's start fresh!\n")
                continue

            if user_input.lower() == "save":
                manager.save_conversation()
                continue

            if user_input.lower() == "stats":
                manager.print_statistics()
                continue

            if user_input.lower() == "history":
                print("\n" + "="*70)
                print("📜 CONVERSATION HISTORY")
                print("="*70)
                for msg in manager.conversation_history:
                    sender = "👤 You" if msg["sender"] == "user" else "🤖 Bot"
                    emotion_info = ""
                    if msg.get("emotion"):
                        conf = f"{msg['confidence']:.0%}" if msg.get('confidence') else ""
                        emotion_info = f" [{msg['emotion']} {conf}]" if conf else f" [{msg['emotion']}]"
                    print(f"{sender}{emotion_info}: {msg['message']}")
                print("="*70 + "\n")
                continue

            # Add user message
            manager.add_message("user", user_input)

            # Generate response
            emotion, confidence, response = generate_response(
                user_input,
                manager.conversation_history
            )

            # Add bot response
            manager.add_message("bot", response, emotion, confidence)

            # Display response
            confidence_bar = "🔴" if confidence < 0.6 else "🟡" if confidence < 0.8 else "🟢"
            print(f"🤖 Bot [{emotion} {confidence_bar} {confidence:.0%}]: {response}\n")

        except KeyboardInterrupt:
            print("\n\n" + "="*70)
            manager.print_statistics()
            manager.save_conversation()
            print("👋 Take care!\n")
            break
        except Exception as e:
            print(f"⚠️ Error: {e}\n")
            continue

# =====================================
# Main
# =====================================

if __name__ == "__main__":
    chat()